In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys

#instant execution, useful for dev
tf.enable_eager_execution()

print(tf.__version__)
print(tf.keras.__version__)
print(pd.__version__)
print(np.__version__)
print (sys.version_info)

1.13.1
2.2.4-tf
0.23.4
1.16.2
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


In [0]:
#load data
from google.colab import drive
drive.mount('/content/drive/')

dftrain=pd.read_csv('/content/drive/My Drive/train.csv')
dfpredict=pd.read_csv('/content/drive/My Drive/test.csv')

print(dftrain.head())
print(dftrain.shape[0])
print(dfpredict.head())
print(dfpredict.shape[0])

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
   Store  Dept        Date  Seasonality  Year      Size  Type_A  Type_B  \
0      1     1  2010-02-05     0.078431   0.0  0.630267       1       0   
1      1     1  2010-02-12     0.098039   0.0  0.630267       1       0   
2      1     1  2010-02-19     0.117647   0.0  0.630267       1       0   
3      1     1  2010-02-26     0.137255   0.0  0.630267       1       0   
4      1     1  2010-03-05     0.156863   0.0  0.630267       1       0   

   Type_C  Temperature      ...       MarkDown3  MarkDown4  MarkDown5  \
0       0     0.454046      ...             0.0        0.0        0.0   
1       0     0.419260      ...             0.0        0.0        0.0   
2       0     0.432259      ...             0.0        0.0        0.0   
3       0     0.493592      ...             0.0        0.0        0.0   
4       0     0.492402      ...             0.0      

In [0]:
#I don't need the Date,Store and Dept columns for NN (but used for submission)
dftrain.drop("Date", axis=1, inplace=True) 
dftrain.drop("Store", axis=1, inplace=True) 
dftrain.drop("Dept", axis=1, inplace=True) 
dfpredict.drop("Date", axis=1, inplace=True) 
dfpredict.drop("Store", axis=1, inplace=True) 
dfpredict.drop("Dept", axis=1, inplace=True) 

print(dftrain.shape[0])
print(dfpredict.shape[0])

421570
115064


In [0]:
#force all types to float32
dftrain = dftrain.astype(np.float32)
dfpredict = dfpredict.astype(np.float32)

#split train data in features and labels
y = dftrain.Weekly_Sales
x = dftrain.drop('Weekly_Sales', axis=1)

#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [0]:
#convert the dataframes to numpy arrays
#y_train = y_train.to_numpy() requires 0.24
y_train = y_train.values
x_train = x_train.values
y_test = y_test.values
x_test = x_test.values
x_predict = dfpredict.values

In [0]:
#here is the NN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=19, activation='relu', kernel_initializer='normal', input_dim=19))
#model.add(Dense(units=25, activation='relu', kernel_initializer='normal', input_dim=14))
#model.add(Dense(7, kernel_initializer='normal', activation='relu'))
#model.add(Dense(units=1, activation='linear'))
model.add(Dense(units=1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error',
              optimizer='Adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

Train on 337256 samples, validate on 84314 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
337256/337256 [==============================] - 14s 41us/sample - loss: 659651320.9790 - acc: 4.0325e-04 - val_loss: 549466306.7418 - val_acc: 4.0325e-04
Epoch 2/25
337256/337256 [==============================] - 14s 43us/sample - loss: 507751655.8662 - acc: 3.9732e-04 - val_loss: 514172819.1012 - val_acc: 4.0325e-04
Epoch 3/25
337256/337256 [==============================] - 13s 39us/sample - loss: 495989466.6744 - acc: 3.9732e-04 - val_loss: 507579539.5103 - val_acc: 4.0325e-04
Epoch 4/25
337256/337256 [==============================] - 13s 40us/sample - loss: 490351743.1779 - acc: 3.9732e-04 - val_loss: 503024823.6115 - val_acc: 4.0325e-04
Epoch 5/25
337256/337256 [==============================] - 13s 40us/sample - loss: 486451650.5808 - acc: 3.9732e-04 - val_loss: 499807822.0966 - val_acc: 4.0325e-04
Epoch 6/25
337256/337256 [==============================] - 13s 39us/sa

In [0]:
# Predict!!
y_predict = model.predict(x_predict)

#for i in range(len(x_predict)):
for i in range(10):
	print("x=%s, y=%s" % (x_predict[i], y_predict[i]))

x=[0.84313726 0.6666667  0.6302673  1.         0.         0.
 0.5731417  0.45791584 0.09010297 0.0516624  0.00153732 0.05394146
 0.00378754 0.9464236  0.27180356 1.         0.         0.
 0.        ], y=[15756.385]
x=[0.8627451  0.6666667  0.6302673  1.         0.         0.
 0.6273343  0.42184368 0.13403085 0.03470573 0.0014669  0.06886396
 0.00821547 0.94660366 0.27180356 1.         0.         0.
 0.        ], y=[15952.325]
x=[0.88235295 0.6666667  0.6302673  1.         0.         0.
 0.5511717  0.39078155 0.11775173 0.00532384 0.00189119 0.01679046
 0.0088097  0.94691074 0.27180356 1.         0.         0.
 0.        ], y=[15474.631]
x=[9.0196079e-01 6.6666669e-01 6.3026732e-01 1.0000000e+00 0.0000000e+00
 0.0000000e+00 5.8147198e-01 3.7024048e-01 3.4586802e-02 2.5760292e-03
 5.0172585e-01 3.1076865e-03 6.3305977e-04 9.4738722e-01 2.7180356e-01
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00], y=[19886.475]
x=[0.92156863 0.6666667  0.6302673  1.         0.         0.
 0.545

In [0]:
#save predictions to csv
from google.colab import drive
drive.mount('/content/drive/')
np.savetxt("/content/drive/My Drive/prediction.csv", y_predict, fmt="%10.5f", delimiter=",")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
